# Change pressure above a threshold
- Running into HUGE pressures adjacent to sinks in CONUS2, particularly in the Great Basin. 
- Here, change a pressure file so that an pressure above a certain threshold will be ~0.1m

### Import Packages
Here we will be using the reading and writing tools that come with the ParFlow tools package.  

In [1]:
import xarray as xr
import numpy as np
import os
from glob import glob
import matplotlib.pyplot as plt
from matplotlib import figure
import pandas as pd
from osgeo import gdal


import parflow as pf
from parflow.tools.fs import get_absolute_path
from parflow.tools.io import write_pfb, read_pfb
from parflow import Run
import parflow.tools.hydrology as hydro

import math
#setting the directory name that we will read our outputs from

# run_dir ='/glade/scratch/tijerina/CONUS2/scaling_runs/spinup_scaling/outputs_r1_0-25_4'
# run_name ='spinup.scaling.48_36'
run_name = 'clm-update_CONUS2'
run_dir = '/glade/scratch/tijerina/CONUS2/scaling_runs/clm-update-run'

 

print(run_dir)

/glade/scratch/tijerina/CONUS2/scaling_runs/clm-update-run


#### Read in the domain properties
- Outputs are from a test simulation of 24 hours from the scaling study

In [2]:
run = Run.from_definition(f'{run_dir}/{run_name}.pfidb')
data = run.data_accessor
nt = len(data.times)
nx = data.shape[2]
ny = data.shape[1]
nz = data.shape[0]
dx = data.dx
dy = data.dy
dz = data.dz

print(nt,nx,ny,nz,dx,dy,dz)

porosity = data.computed_porosity 
specific_storage = data.specific_storage 
#mannings = pf.read_pfb(f'{run_dir}/spinup.scaling.48_36.out.n.pfb') #run.Mannings.Geom.domain.Value
mannings = data.mannings

## remove input filenames for TopoSlopes to force the data accessor to read the output slopes
## this fixes a windows issue
run.TopoSlopesX.FileName = None
run.TopoSlopesY.FileName = None

slopex = data.slope_x 
slopey = data.slope_y 
mask = data.mask

# formatting the mask so that values outside the domain are NA and inside the domain are 1
# check with mask that has 0 and 1
nanmask=mask.copy()
#nanmask[nanmask == 0] = 'NaN' ---> Use this for NaNs np.nan
nanmask[nanmask > 0] = 1

Solver: Field BinaryOutDir is not part of the expected schema <class 'parflow.tools.database.generated.Solver'>
Solver.OverlandKinematic: Field SeepageOne is not part of the expected schema <class 'parflow.tools.database.generated.OverlandKinematic'>
Solver.OverlandKinematic: Field SeepageTwo is not part of the expected schema <class 'parflow.tools.database.generated.OverlandKinematic'>
  - nt
  - sw_ini
  - hkdepth
  - wtfact
  - trsmx0
  - smpmax
  - pondmx
97 4442 3256 10 1000.0 1000.0 [2.0e+02 1.0e+02 5.0e+01 2.5e+01 1.0e+01 5.0e+00 1.0e+00 6.0e-01 3.0e-01
 1.0e-01]


#### Read a pressure output file

In [3]:
#read in pressure file into a 3D NParray
pressure_array = pf.read_pfb(f'{run_dir}/{run_name}.out.press.00024.pfb') * nanmask


In [4]:
print(pressure_array.shape)
# get one layer of pressure array
pressure9 = pressure_array[9,:,:]
pressure9.shape

(10, 3256, 4442)


(3256, 4442)

In [5]:
print(pressure9.max())
print(pressure9.min())

34526.849718634214
-21.051453299122553


In [6]:
print(pressure_array[9,:,:].max())
print(pressure_array[9,:,:].min())
print("")
print(pressure_array[8,:,:].max())
print(pressure_array[8,:,:].min())
print("")
print(pressure_array[7,:,:].max())
print(pressure_array[7,:,:].min())
print("")
print(pressure_array[6,:,:].max())
print(pressure_array[6,:,:].min())
print("")
print(pressure_array[5,:,:].max())
print(pressure_array[5,:,:].min())
print("")
print(pressure_array[4,:,:].max())
print(pressure_array[4,:,:].min())
print("")
print(pressure_array[3,:,:].max())
print(pressure_array[3,:,:].min())
print("")
print(pressure_array[2,:,:].max())
print(pressure_array[2,:,:].min())
print("")
print(pressure_array[1,:,:].max())
print(pressure_array[1,:,:].min())
print("")
print(pressure_array[0,:,:].max())
print(pressure_array[0,:,:].min())

34526.849718634214
-21.051453299122553

34438.02870332217
-20.85773043024164

34242.26094977139
-20.462203402147708

33456.776044840255
-19.885036429916642

30281.47770804876
-18.21005517573819

25366.635024381565
-20.429080848458245

4585.599982704118
-11.402365957406777

723.0528951514855
-9.332261311209523

624.97916925624
-5.423892222591665

780.014048685796
-4.91754079942326


In [7]:
above_threshold = np.where(pressure9 > 15)
np.count_nonzero(above_threshold)

278

In [8]:
pressure_array[9,...][pressure_array[9,...]>15]=0.1 # change any pressures > 15 m to 0.1 m

In [9]:
print(pressure_array[9,...].max())
print(pressure_array[9,...].min())

14.81513210158111
-21.051453299122553


In [10]:
print(pressure9.max())
print(pressure9.min())

14.81513210158111
-21.051453299122553


### Create new pfb with high pressures set to a lower value

In [11]:
write_pfb('down_scaled_pressure_CLM-update.pfb',pressure_array)

In [12]:
# Check below a threshold for fun / why are these pressures so low??
below_threshold = np.where(pressure9 < -13000)
np.count_nonzero(below_threshold)

0

## Calculate overlandflow

In [ ]:
overland_flow= hydro.calculate_overland_flow_grid(pressure_array, slopex, slopey, mannings, dx, dy, mask = nanmask)/3600
    # divide by 3600 to go from [m^3/h] to [m^3/s]

In [ ]:
plt.figure(figsize = (9,7))
plt.imshow(overland_flow, cmap="Blues", origin='lower', vmin = 0, vmax = 25)
plt.colorbar()
plt.title('Overland Flow for pressure file')
#plt.savefig('overlandflow_hour_cms.png', dpi=1000)
plt.show()


In [ ]:
# Checking min/max/shape (cms)
print('Hourly overland flow (cms): ')
print(f'Shape: {overland_flow.shape}')
print(f'Max: {overland_flow.max()}')
print(f'Min: {overland_flow.min()}')
print(" ")
print('Daily overland flow (cms... not m^3/d?): ')
print(f'Shape: {daily_overland_flow.shape}')
print(f'Max: {daily_overland_flow.max()}')
print(f'Min: {daily_overland_flow.min()}')

### Create pressure GeoTiff for GIS 

In [7]:
### CREATE GEOTIFF
# this file is needed for the geotransform and projection below
file = '1km_CONUS2_landcover_IGBP.tif'

# open tiff mask
lc_tif = gdal.Open(file)

type(lc_tif)

osgeo.gdal.Dataset

In [8]:
# Flip df to make tif
pressure_array9_flip = np.flipud(pressure9) #pressure9

# Create geotiff of CLM variables for QGIS
dst_filename = 'bad_pressures.tiff' #'Pressure_top_layer_forThreshold.tiff'
x_pixels = 4442  # number of grid cells in x
y_pixels = 3256  # number of grid cells in y
driver = gdal.GetDriverByName('GTiff')
dataset = driver.Create(dst_filename,x_pixels, y_pixels, 1,gdal.GDT_Float32)
dataset.GetRasterBand(1).WriteArray(pressure_array9_flip) ### create tif with array you want to use

# Add GeoTranform and Projection information to newly created tif file
geotrans=lc_tif.GetGeoTransform()  #get GeoTranform from existing tif file (here 'conus2tiff')
proj=lc_tif.GetProjection() #you can import or get from a existing tif   
dataset.SetGeoTransform(geotrans)
dataset.SetProjection(proj)
dataset.FlushCache()
dataset=None